In [99]:
import sys
import pandas as pd
import requests
import re
import os
import time
import codecs
import pickle
from pandas import Series, DataFrame
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 오늘의 웹소설
---
http://novel.naver.com/webnovel/weekdayList.nhn

### Parameters

In [2]:
base_url = 'http://novel.naver.com'
genre_base_url = 'http://novel.naver.com/webnovel/genre.nhn?'

In [3]:
#로맨스, 로맨스판타지, 판타지, 무협,미스터리,역사/전쟁, 라이트노벨, 퓨전
# 장르 이름 : [url_num, 'url_sep']
genre_item_list = {
    'romance' : [101, 'rom'], 
    'romancefantasy' : [109, 'rof'], 
    'fantasy' : [102, 'sff'], 
    'action' :[103, 'hro'],
    'mystery' : [104,'mth'],
    'history&war' :[105, 'his'], 
    'lightnovel' :[106, 'lno'], 
    'fusion' : [108, 'fus']
} 

### Functions

In [4]:
# 장르별 소설 url dict 가져옴
def get_novel_url_dict(genre_name, finished):
    novel_dict = {} # 소설 제목 : 소설 url 형태로 return
    
    genre_item = genre_item_list[genre_name]
    genre_num = genre_item[0]
    url_sep = genre_item[1]
    
    for page_num in range(1,20):
        params = {
            'genre' : genre_num,
            'page' : page_num,
        }

        if finished: # 완결작일경우 url 다름
            params['order'] = 'Read'
            params['finish']= 'true'

        r = requests.get('http://novel.naver.com/webnovel/genre.nhn?', params=params)
        crawler = BeautifulSoup(r.content, 'html.parser')
        
        class_temp = 'list_type1 v2 NE=a:lst_' + url_sep
        novel_list = crawler.find('ul', {'class' :class_temp}).findAll('li')
        for novel in novel_list:
            url = novel.a.attrs['href'] # url
            title = novel.find('p', {'class': 'subj v3'}).contents[0] # title
            if title in novel_dict:
                print(str(page_num) + '페이지에서 완료')
                return novel_dict
            else:
                novel_dict[title] = base_url + url
        time.sleep(0.05)
    return novel_dict

In [5]:
#유효한 response인지 검사
def valid_status(response):
    if response.status_code == 200:
        return True
    else :
        return False

In [6]:
TITLE_TOKEN = '<title>'
TALK_TOKEN = '<talk>'
QUOTES = '[“”‘’-]' # will be stripped

# url로부터 내용 읽어서 파싱
def get_content_from_url(content_page_url):
    content = ''
    r = requests.get(content_page_url)
    if valid_status(r):
        pass
    else:
        print("오류 발생 : {}".format(content_page_url))
        return ''
    
    crawler = BeautifulSoup(r.content, 'html.parser')
    
    title = crawler.find('h2', {'class': 'detail_view_header'}).contents[0] # 몇화 제목
    content += TITLE_TOKEN + title + '\r\n\r\n'
    
    line_list = crawler.find('div', {'class': 'detail_view_content ft15'}).find_all('p')
    for line in line_list :
        if line.has_attr('class') == False: # general sentence
            processed_sentence = line.get_text().strip()
            content += processed_sentence
        elif 'talk' in line['class']: # talk
            processed_talk = line.get_text().strip().strip(QUOTES).strip()
            content += TALK_TOKEN + processed_talk
        elif 'pic' in line['class']: # picture
            pass
        else:
            print('기타 태그 오류 발생')
            print('line')
        content = content + '\r\n'
    time.sleep(0.05)
    
    return content

In [26]:
# url_list로부터 읽어와서 텍스트 파일에 저장
def crawl_and_write(genre, novel_url_dict, finished):
    if finished:
        finish_string = '완결'
    else:
        finish_string = '미완결'
    print('{} {} 장르 총 url {}개'.format(finish_string, genre, len(novel_url_dict)))
    print('----------------------------------------')
    check_index = 0

    for title, url in novel_url_dict.items():
        check_index = check_index + 1
        file_name = title + '.txt' # filename
        file_name = re.sub('[\/:*?"<>|]','',file_name) # file명 형식에 맞게 수정

        r = requests.get(url)
        crawler = BeautifulSoup(r.content, 'html.parser')

        total = int(crawler.find("span", {"class" : "total"}).get_text()[1:-1]) #전체 회차
        if total == 0: # 회차 없으면 멈춤
            break;
        page_limit = total // 10 #한 페이지에 10개씩
        if total % 10 != 0:
            page_limit += 1
#         print(total)
#         print(page_limit)
        
        # 완결/미완결에 따라 분기
        if finished:
            iterate_range = range(1, page_limit+1)
        else:
            iterate_range = range(1, page_limit+1)
        
        novel_content_list = []
        for page_num in iterate_range:
            r = requests.get(url, params = {'page': page_num})
    #         print(r.url)
            crawler = BeautifulSoup(r.content, "html.parser")
            page_novel_content_list = crawler.find("ul", {"class": "list_type2 v3 NE=a:lst"}).findAll("li") # 이 페이지의 모든 소설 리스트
            novel_content_list.extend(page_novel_content_list)
            
        if not finished:
            novel_content_list.reverse()
            
        write_string = ''
        for novel_content in novel_content_list:
            content_url = novel_content.a.attrs['href']
            content_page_url = base_url + content_url #소설 내용 url
            write_string  = write_string + get_content_from_url(content_page_url)

        with codecs.open(os.path.join(directory, file_name), 'w', encoding = 'utf-8') as f: #file에 write
            f.write(write_string)
        print('{} : 저장 성공'.format(title))
        
    # 진행 상황 출력
    if check_index % 10 == 0:
        print("** 현재 " + str(check_index) +"개 진행중...")

### Let's Crawl

In [17]:
base_dir = r'C:\Users\kwj96\Desktop\YBigta\웹소설 공모전\webnovel_data'
for genre in genre_item_list.keys():
    directory = os.path.join(base_dir, genre)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    for finished in True,False: #완결, 미완결
        if (genre == 'history&war') & (finished == True):
            continue
        novel_url_dict = get_novel_url_dict(genre, finished) #장르별 소설 url list
        crawl_and_write(genre, novel_url_dict, finished) #파일에 저장
        print('\n')
    print('\n')

2페이지에서 완료
미완결 history&war 장르 총 url 1개
----------------------------------------
연못에 핀 목화 - 송경별곡 : 저장 성공




2페이지에서 완료
완결 lightnovel 장르 총 url 10개
----------------------------------------
이사님의 취미생활 : 저장 성공
마성의 가정부 : 저장 성공
이 집사님의 사랑법 : 저장 성공
협박연애 : 저장 성공
형의 그녀 : 저장 성공
납치 감금에서 시작되는 우리들의 사바트 : 저장 성공
애유기 : 저장 성공
사또의 여자가 되겠나이다 : 저장 성공
여왕의 아이돌 : 저장 성공
앨리스 드라이브 : 저장 성공
** 현재 10개 진행중...


2페이지에서 완료
미완결 lightnovel 장르 총 url 3개
----------------------------------------
용왕님의 셰프가 되었습니다 : 저장 성공
전설의 공돌이 : 저장 성공
편의점에서 그녀와 창세하고 있습니다 : 저장 성공




2페이지에서 완료
완결 fusion 장르 총 url 11개
----------------------------------------
백의서소 : 저장 성공
우아한 환생 : 저장 성공
광야의 야수들 : 저장 성공
드라마입니까 : 저장 성공
레이븐: 여왕의 수호 목걸이 : 저장 성공
고스트 게이트 : 저장 성공
채널 나인 : 저장 성공
개경 세원록 : 저장 성공
빈틈없이 고요하게 : 저장 성공
언어영역 완전정복 : 저장 성공
** 현재 10개 진행중...
계와 과학자 : 저장 성공


2페이지에서 완료
미완결 fusion 장르 총 url 1개
----------------------------------------
조선공주실록 : 저장 성공






# 오늘의 웹소설 Metadata
---

### Parameters

In [56]:
base_url = 'http://novel.naver.com'
genre_base_url = 'http://novel.naver.com/webnovel/genre.nhn?'

In [57]:
#로맨스, 로맨스판타지, 판타지, 무협,미스터리,역사/전쟁, 라이트노벨, 퓨전
# 장르 이름 : [url_num, 'url_sep']
genre_item_list = {
    'romance' : [101, 'rom'], 
    'romancefantasy' : [109, 'rof'], 
    'fantasy' : [102, 'sff'], 
    'action' :[103, 'hro'],
    'mystery' : [104,'mth'],
    'history&war' :[105, 'his'], 
    'lightnovel' :[106, 'lno'], 
    'fusion' : [108, 'fus']
} 

### Functions

In [97]:
# metadata
# {'10도, 우리 연애의 온도차': ['설래인', 483047, 8, 9.97, 24066]}
# {제목 : [작가,novelid, 총 회차, 평점, 관심수]}
def get_metadata_dict(genre_name, finished):
    metadata_dict = {}
    
    genre_item = genre_item_list[genre_name]
    genre_num = genre_item[0]
    url_sep = genre_item[1]
    
    for page_num in range(1,20):
        params = {
            'genre' : genre_num,
            'page' : page_num,
        }

        if finished: # 완결작일경우 url 다름
            params['order'] = 'Read'
            params['finish']= 'true'

        r = requests.get('http://novel.naver.com/webnovel/genre.nhn?', params=params)
        crawler = BeautifulSoup(r.content, 'html.parser')
        
        class_temp = 'list_type1 v2 NE=a:lst_' + url_sep
        novel_list = crawler.find('ul', {'class' :class_temp}).findAll('li')
        for novel in novel_list:
            url = novel.a.attrs['href'] # url
            novel_id = url.split('=')[-1] #novel_id
            title = novel.find('p', {'class': 'subj v3'}).contents[0] # title
            title = re.sub('[\/:*?"<>|]','',title) #특수문자 제거
            author = novel.find('span',{'class':'ellipsis'}).get_text() # author
            total_episode = novel.find('span',{'class':'num_total'}).get_text()[2:-1] # total_episode
            star_rating = novel.find('span',{'class':'score_area'}).get_text()[2:] #별점
            attention_rating = novel.find('span',{'class':'info_text'}).get_text()[2:].replace(',', '') #관심 개수
            if attention_rating[-1] == '만':
                attention_rating = float(attention_rating[:-1]) * 10000
            
            if title in metadata_dict:
                print(str(page_num) + '페이지에서 완료')
                return metadata_dict
            else:
                metadata_dict[title] = [author, int(novel_id), int(total_episode), float(star_rating), int(attention_rating)]
        time.sleep(0.05)
    return metadata_dict

### Let's Crawl

In [98]:
metadata_dict = {}
for genre in genre_item_list.keys():
    for finished in True,False: #완결, 미완결
        if (genre == 'history&war') & (finished == True):
            continue
        metadata_dict.update(get_metadata_dict(genre, finished))
    print('** %s 장르 완료' % genre)
metadata_dict

10페이지에서 완료
4페이지에서 완료
** romance 장르 완료
3페이지에서 완료
2페이지에서 완료
** romancefantasy 장르 완료
3페이지에서 완료
2페이지에서 완료
** fantasy 장르 완료
3페이지에서 완료
2페이지에서 완료
** action 장르 완료
3페이지에서 완료
2페이지에서 완료
** mystery 장르 완료
2페이지에서 완료
** history&war 장르 완료
2페이지에서 완료
2페이지에서 완료
** lightnovel 장르 완료
2페이지에서 완료
2페이지에서 완료
** fusion 장르 완료


{'10도, 우리 연애의 온도차': ['설래인', 483047, 8, 9.97, 24066],
 '12시의 신데렐라': ['백우시', 554031, 5, 9.94, 12010],
 '13월의 첫사랑': ['서별아', 319785, 5, 9.95, 16520],
 '19세기 비망록': ['AuthenticA', 73177, 5, 9.91, 12117],
 '30인의 회귀자': ['이성현', 562058, 184, 9.98, 7348],
 'The Man': ['정예인', 153310, 5, 9.71, 13206],
 '[SYSTEM]판타지를 로딩합니다': ['웹소설 작가', 657176, 56, 9.85, 1993],
 '가르쳐 주세요': ['플아다', 523286, 8, 9.95, 37487],
 '가면 속 그대': ['손작가', 272508, 5, 9.87, 17767],
 '갓 오브 하이스쿨 - 이클립스': ['블로D', 398094, 81, 9.76, 5269],
 '강호제일 해결사': ['장담', 215396, 10, 9.82, 11105],
 '개경 세원록': ['박소헌', 538926, 11, 9.92, 4360],
 '개미들': ['아진', 50518, 5, 9.77, 3965],
 '개의 주인': ['유세라', 686274, 55, 9.97, 10499],
 '갬블링 1945': ['박스오피스', 693362, 42, 9.94, 2110],
 '거짓말 퍼즐': ['김은정', 348264, 7, 9.93, 3983],
 '거짓말의 법칙': ['정이준', 466381, 5, 9.96, 20257],
 '검은 늑대가 나를 부르면': ['임혜', 699567, 37, 9.97, 18994],
 '검이여 노래하라': ['홍정훈', 12, 20, 9.82, 21808],
 '결혼은 운명이다': ['Lunar 이지연', 153292, 6, 9.94, 46328],
 '경천동지': ['이윤아', 374461, 10, 9.76, 7809],
 '계와 과학자': 

### Save metadata as pickle file

In [101]:
import pickle

with open('metadata_dict.pickle', 'wb') as handle:
    pickle.dump(metadata_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [100]:
# load pickle file
# with open('metadata_dict.pickle', 'rb') as handle:
#     dic = pickle.load(handle)